In [8]:
pip install pandas geopandas matplotlib folium pykml

Note: you may need to restart the kernel to use updated packages.


In [16]:
import folium
import pandas as pd
import os
from folium import FeatureGroup, LayerControl, GeoJson, Marker, PolyLine, Map
from shapely.geometry import Polygon
import geopandas as gpd
from pykml import parser

# Crear el mapa base
m = folium.Map(location=[-16.3989, -71.5350], zoom_start=12)

# === Capa 1: Puestos de Salud ===
fg_salud = FeatureGroup(name="Puestos de Salud")
df_puntos = pd.read_csv("SPATIAL_DATA_AQP/Health_centers_AQP/Puestos_de_salud_APQ_12ene2024.csv", sep=";")

for _, row in df_puntos.iterrows():
    Marker(
        location=[row['lat'], row['long']],
        popup=row['name'],
        icon=folium.Icon(color="darkred", icon="plus-sign")
    ).add_to(fg_salud)

fg_salud.add_to(m)

# === Capa 2: Torrenteras ===
fg_torrenteras = FeatureGroup(name="Torrenteras")
df_lines = pd.read_csv("SPATIAL_DATA_AQP/Water_channels_AQP/Hidrografia_AQP_18ene2024.csv", sep=";")
df_lines['lat'] = pd.to_numeric(df_lines['lat'], errors='coerce')
df_lines['long'] = pd.to_numeric(df_lines['long'], errors='coerce')
df_lines = df_lines.dropna(subset=['lat', 'long'])

for ident, grupo in df_lines.groupby('ident'):
    coords = list(zip(grupo['lat'], grupo['long']))
    PolyLine(
        locations=coords,
        color='#8303fb',
        weight=2,
        tooltip=ident
    ).add_to(fg_torrenteras)

fg_torrenteras.add_to(m)


# === Capa 3: Delimitación de Clusters ===
fg_clusters = FeatureGroup(name="Delimitación Cluster")
folder_path = "SPATIAL_DATA_AQP/Delimitation_AQP/Delimitacion_cluster/"

for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        filepath = os.path.join(folder_path, filename)
        df_poly = pd.read_csv(filepath, sep=";")

        # Limpiar datos
        df_poly['lat'] = pd.to_numeric(df_poly['lat'], errors='coerce')
        df_poly['long'] = pd.to_numeric(df_poly['long'], errors='coerce')
        df_poly = df_poly.dropna(subset=['lat', 'long'])

        # Agrupar por 'ident'
        for ident, grupo in df_poly.groupby('ident'):
            coords = list(zip(grupo['lat'], grupo['long']))

            # Crear objeto GeoJson para tooltip interactivo
            polygon = {
                "type": "Feature",
                "properties": {"name": ident},
                "geometry": {
                    "type": "Polygon",
                    "coordinates": [[ [lon, lat] for lat, lon in coords ]]  # GeoJSON usa lon,lat
                }
            }

            GeoJson(
                data=polygon,
                style_function=lambda feature: {
                    "fillColor": "#51f9ee",
                    "color": "black",        # Borde negro más visible
                    "weight": 2.5,           # Grosor del borde aumentado
                    "fillOpacity": 0.2       # Relleno más tenue para que el borde resalte
                },
                tooltip=folium.GeoJsonTooltip(fields=["name"], aliases=["Cluster"])
            ).add_to(fg_clusters)

fg_clusters.add_to(m)

# === Capa 4: Socio-economico ===
fg_socio_economic = FeatureGroup(name="Niveles Socioeconómicos")
# Directorio donde están los CSV
folder_path = 'SPATIAL_DATA_AQP/Economic_income_AQP/Economic_income_cluster/'

# Colores para cada nivel (puedes ajustar esto según tus necesidades)
colores = {
    'A': '#1a9850',  # Verde oscuro (nivel más alto)
    'B': '#91cf60',  # Verde claro
    'C': '#d9ef8b',  # Amarillo
    'D': '#fc8d59',  # Naranja
    'E': '#d73027',  # Rojo (nivel más bajo)
}

# Leer todos los archivos CSV del directorio
for archivo in os.listdir(folder_path):
    if archivo.endswith('.csv'):
        ruta = os.path.join(folder_path, archivo)
        df = pd.read_csv(ruta, sep=';')

        # Limpia espacios en blanco en los nombres de columnas
        df.columns = df.columns.str.strip()

        # Asegurar que las columnas de lat/lon están como texto (por si hay espacios vacíos)
        df['lat'] = df['lat'].astype(str).str.strip()
        df['long'] = df['long'].astype(str).str.strip()

        for ident, grupo in df.groupby('ident'):
            puntos = []

            for _, fila in grupo.iterrows():
                try:
                    lat = float(fila['lat'])
                    lon = float(fila['long'])
                    puntos.append((lon, lat))
                except ValueError:
                    continue  # Ignorar filas vacías

            if len(puntos) < 3:
                continue  # Evitar errores por polígonos inválidos

            poligono = Polygon(puntos)

            nivel = grupo['ingreso_economico'].iloc[0]
            color = colores.get(nivel, 'gray')

            geojson = gpd.GeoDataFrame(
                [{'geometry': poligono, 'ingreso_economico': nivel}],
                crs='EPSG:4326'
            )

            folium.GeoJson(
                geojson,
                style_function=lambda feature, color=color: {
                    'fillColor': color,
                    'color': color,
                    'weight': 2,
                    'fillOpacity': 0.4
                },
                tooltip=folium.GeoJsonTooltip(
                    fields=['ingreso_economico'],
                    aliases=['Nivel:']
                )
            ).add_to(fg_socio_economic)

# Agregar capa al mapa
fg_socio_economic.add_to(m)

# === Capa 5: Carreteras ===
fg_roads = FeatureGroup(name="Carreteras")
df_roads = pd.read_csv("SPATIAL_DATA_AQP/Access_roads_AQP/Roads/Carreteras_AQP_14ene2024.csv", sep=";")
df_roads['lat'] = pd.to_numeric(df_roads['lat'], errors='coerce')
df_roads['long'] = pd.to_numeric(df_roads['long'], errors='coerce')
df_roads = df_roads.dropna(subset=['lat', 'long'])

for type, grupo in df_roads.groupby('type'):
    coords = list(zip(grupo['lat'], grupo['long']))
    PolyLine(
        locations=coords,
        color='#581845',
        weight=3,
        tooltip=type
    ).add_to(fg_roads)

fg_roads.add_to(m)

# === Capa 6: Carreteras ===
fg_train_line = FeatureGroup(name="Lineas de tren")
df_train_line = pd.read_csv("SPATIAL_DATA_AQP/Access_roads_AQP/Train_line/Linea_tren_AQP_14ene2024.csv", sep=";")
df_train_line['lat'] = pd.to_numeric(df_train_line['lat'], errors='coerce')
df_train_line['long'] = pd.to_numeric(df_train_line['long'], errors='coerce')
df_train_line = df_train_line.dropna(subset=['lat', 'long'])

for ident, grupo in df_train_line.groupby('ident'):
    coords = list(zip(grupo['lat'], grupo['long']))
    PolyLine(
        locations=coords,
        color='#6F4E37',
        weight=3,
        tooltip=ident
    ).add_to(fg_train_line)

fg_train_line.add_to(m)

# ===== Capa de casos de rabia por año =================
# Colores para distinguir capas
colores = [
    'red', 'blue', 'green', 'purple', 'orange',
    'lightblue', 'darkgreen', 'pink', 'cadetblue'
]

# Carpeta con archivos .kml
folder_path = "/home/pantro/Descargas/KML_casos_rabia/"
kml_files = sorted([f for f in os.listdir(folder_path) if f.endswith(".kml")])  # Ordenar alfabéticamente

for idx, filename in enumerate(kml_files):
    filepath = os.path.join(folder_path, filename)
    with open(filepath, 'r', encoding='utf-8') as f:
        root = parser.parse(f).getroot()
        placemarks = root.Document.findall(".//{http://www.opengis.net/kml/2.2}Placemark")

        # Asignar color por archivo
        color = colores[idx % len(colores)]

        count = 0
        group_name = f"{filename.replace('.kml', '')} ({len(placemarks)} casos)"
        group = FeatureGroup(name=group_name)

        for pm in placemarks:
            coords_tag = pm.find(".//{http://www.opengis.net/kml/2.2}coordinates")
            if coords_tag is not None:
                coords_text = coords_tag.text.strip()
                try:
                    lon, lat, *_ = map(float, coords_text.split(','))
                    folium.Marker(
                            location=[lat, lon],
                            popup=filename.replace(".kml", ""),
                            icon=folium.Icon(color=color, icon="info-sign")
                        ).add_to(group)
                    
                    count += 1
                except ValueError:
                    continue  # Coord inválida

        if count > 0:
            group.add_to(m)

# === Control de capas ===
LayerControl(collapsed=False).add_to(m)

# === Guardar mapa ===
m.save("mapa_completo_arequipa.html")
print("Finalizo la creacion del mapa")


Finalizo la creacion del mapa
